# Holdout Validation

You're probably used to 80-20 train/test data splitting. But, here, we're going to do 50-50 train-test one way and then train-test using the complementary data.. So we'll end up with two models and two error numbers which we'll then average. 

The ultimate goal DQ hasn't told me yet :)